In [1]:
import pandas as pd
import numpy as np

In [2]:
daegu = pd.read_csv('accidentInfoList_대구.csv', header=1)

In [3]:
daegu.to_csv('Daegu.csv', index = False)

In [4]:
daegu = pd.read_csv('Daegu.csv')

# 결측치 처리
- 피해운전자 차종, 피해운전자 성별, 피해운전자 연령, 피해운전자 상해정도 -> 기타불명/미분류로 채우기

In [5]:
daegu.isnull().sum()

사고번호            0
사고일시            0
요일              0
시군구             0
사고내용            0
사망자수            0
중상자수            0
경상자수            0
부상신고자수          0
사고유형            0
법규위반            0
노면상태            0
기상상태            0
도로형태            0
가해운전자 차종        0
가해운전자 성별        0
가해운전자 연령        0
가해운전자 상해정도      0
피해운전자 차종      210
피해운전자 성별      210
피해운전자 연령      210
피해운전자 상해정도    210
dtype: int64

- 결측치 생기는 행 확인 -> 차량단독사고인 경우 결측치가 생김

In [6]:
a = daegu[daegu.isnull().any(axis = 1)]

In [7]:
a['사고유형'].value_counts()

사고유형
차량단독 - 전도전복 - 전도     82
차량단독 - 기타            73
차량단독 - 공작물충돌         49
차량단독 - 도로외이탈 - 추락     3
차량단독 - 도로외이탈 - 기타     2
차량단독 - 주/정차차량 충돌      1
Name: count, dtype: int64

- 결측치 채우기

In [8]:
daegu['피해운전자 차종'].fillna('기타불명', inplace= True)
daegu['피해운전자 성별'].fillna('기타불명', inplace= True)
daegu['피해운전자 연령'].fillna('미분류', inplace= True)
daegu['피해운전자 상해정도'].fillna('기타불명', inplace= True)

# 종속변수현 만들기

# EPDO 종속변수 계산
- EPDO = (사망자수 x 12) + (중상자수 x 6) + (경상자수 x 3) + (부상신고자수) 

In [9]:
daegu['EPDO'] = (daegu['사망자수'] * 12) + (daegu['중상자수'] * 6) + (daegu['경상자수'] * 3) + (daegu['부상신고자수'])

- 종속변수현 생성 후 만들었던 독립변수 제거

In [10]:
daegu.drop(['사망자수', '중상자수', '경상자수', '부상신고자수'], axis = 1, inplace = True)

# 가해운전자 차종은 모두 이륜차라 제거

In [11]:
daegu.drop(['가해운전자 차종'], axis = 1, inplace = True)

# 사고일시 연,월,일,시간대로 변환

In [12]:
daegu['사고일시'] = pd.to_datetime(daegu['사고일시'], format='%Y년 %m월 %d일 %H시')

In [13]:
daegu['year'] = daegu['사고일시'].dt.year
daegu['month'] = daegu['사고일시'].dt.month
daegu['day'] = daegu['사고일시'].dt.day
daegu['사고시간대'] = daegu['사고일시'].dt.hour

In [14]:
daegu.drop(['사고일시'], axis = 1, inplace = True)

# 사고유형, 도로형태 분할

In [15]:
daegu['사고유형_1'] = daegu['사고유형'].str.split('-').str[0]
daegu['사고유형_2'] = daegu['사고유형'].str.split('-').str[1]
daegu['도로형태_1'] = daegu['도로형태'].str.split('-').str[0]
daegu['도로형태_2'] = daegu['도로형태'].str.split('-').str[1]

In [16]:
daegu.drop(['사고유형'], axis = 1, inplace = True)
daegu.drop(['도로형태'], axis = 1, inplace = True)

# 연령대 구분

In [17]:
def 연령대_변환(연령):
    # 연령이 '20세 이하'인 경우
    if 연령 == '20세 이하':
        return '20세 이하'
    
    # '세'가 포함된 경우 처리
    elif '세' in 연령:
        # 숫자만 추출
        나이 = int(''.join(filter(str.isdigit, 연령)))
        
        # 연령대를 조건에 맞게 반환
        if 나이 < 20:
            return '20세 이하'
        elif 나이 < 30:
            return '20대'
        elif 나이 < 40:
            return '30대'
        elif 나이 < 50:
            return '40대'
        elif 나이 < 60:
            return '50대'
        elif 나이 < 70:
            return '60대'
        elif 나이 < 80:
            return '70대'
        else:
            return '80대 이상'
    else:
        return '미분류'

daegu['가해운전자 연령대'] = daegu['가해운전자 연령'].apply(연령대_변환)
daegu['피해운전자 연령대'] = daegu['피해운전자 연령'].apply(연령대_변환)

- 기존 연령 열 삭제

In [18]:
daegu.drop(['가해운전자 연령', '피해운전자 연령'], axis = 1, inplace = True)

In [19]:
daegu.head()

,사고번호,요일,시군구,사고내용,법규위반,노면상태,기상상태,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,...,year,month,day,사고시간대,사고유형_1,사고유형_2,도로형태_1,도로형태_2,가해운전자 연령대,피해운전자 연령대
0,2021010100100174,금요일,대구광역시 남구 대명동,경상사고,안전운전불이행,건조,맑음,남,상해없음,승용,...,2021,1,1,18,차대차,추돌,교차로,교차로부근,20대,20대
1,2021010100100258,금요일,대구광역시 중구 교동,부상신고사고,신호위반,건조,맑음,남,부상신고,승용,...,2021,1,1,22,차대차,측면충돌,교차로,교차로안,40대,60대
2,2021010200100088,토요일,대구광역시 서구 비산동,경상사고,안전운전불이행,건조,맑음,여,경상,승용,...,2021,1,2,11,차대차,측면충돌,교차로,교차로안,60대,50대
3,2021010200100338,토요일,대구광역시 수성구 범물동,부상신고사고,직진우회전진행방해,건조,맑음,남,부상신고,승용,...,2021,1,2,19,차대차,측면충돌,교차로,교차로안,40대,60대
4,2021010200100375,토요일,대구광역시 북구 침산동,경상사고,신호위반,건조,맑음,남,상해없음,보행자,...,2021,1,2,20,차대사람,횡단중,교차로,교차로부근,20대,20대


# 시군구 열 구, 동으로 분할

In [20]:
daegu['구'] = daegu['시군구'].str.split(' ').str[1]
daegu['동'] = daegu['시군구'].str.split(' ').str[2]

In [21]:
daegu.head()

,사고번호,요일,시군구,사고내용,법규위반,노면상태,기상상태,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,...,day,사고시간대,사고유형_1,사고유형_2,도로형태_1,도로형태_2,가해운전자 연령대,피해운전자 연령대,구,동
0,2021010100100174,금요일,대구광역시 남구 대명동,경상사고,안전운전불이행,건조,맑음,남,상해없음,승용,...,1,18,차대차,추돌,교차로,교차로부근,20대,20대,남구,대명동
1,2021010100100258,금요일,대구광역시 중구 교동,부상신고사고,신호위반,건조,맑음,남,부상신고,승용,...,1,22,차대차,측면충돌,교차로,교차로안,40대,60대,중구,교동
2,2021010200100088,토요일,대구광역시 서구 비산동,경상사고,안전운전불이행,건조,맑음,여,경상,승용,...,2,11,차대차,측면충돌,교차로,교차로안,60대,50대,서구,비산동
3,2021010200100338,토요일,대구광역시 수성구 범물동,부상신고사고,직진우회전진행방해,건조,맑음,남,부상신고,승용,...,2,19,차대차,측면충돌,교차로,교차로안,40대,60대,수성구,범물동
4,2021010200100375,토요일,대구광역시 북구 침산동,경상사고,신호위반,건조,맑음,남,상해없음,보행자,...,2,20,차대사람,횡단중,교차로,교차로부근,20대,20대,북구,침산동


# 모델링 코드

In [22]:
daegu

,사고번호,요일,시군구,사고내용,법규위반,노면상태,기상상태,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,...,day,사고시간대,사고유형_1,사고유형_2,도로형태_1,도로형태_2,가해운전자 연령대,피해운전자 연령대,구,동
0,2021010100100174,금요일,대구광역시 남구 대명동,경상사고,안전운전불이행,건조,맑음,남,상해없음,승용,...,1,18,차대차,추돌,교차로,교차로부근,20대,20대,남구,대명동
1,2021010100100258,금요일,대구광역시 중구 교동,부상신고사고,신호위반,건조,맑음,남,부상신고,승용,...,1,22,차대차,측면충돌,교차로,교차로안,40대,60대,중구,교동
2,2021010200100088,토요일,대구광역시 서구 비산동,경상사고,안전운전불이행,건조,맑음,여,경상,승용,...,2,11,차대차,측면충돌,교차로,교차로안,60대,50대,서구,비산동
3,2021010200100338,토요일,대구광역시 수성구 범물동,부상신고사고,직진우회전진행방해,건조,맑음,남,부상신고,승용,...,2,19,차대차,측면충돌,교차로,교차로안,40대,60대,수성구,범물동
4,2021010200100375,토요일,대구광역시 북구 침산동,경상사고,신호위반,건조,맑음,남,상해없음,보행자,...,2,20,차대사람,횡단중,교차로,교차로부근,20대,20대,북구,침산동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,2023123000100440,토요일,대구광역시 북구 동천동,중상사고,신호위반,젖음/습기,비,남,상해없음,자전거,...,30,20,차대차,측면충돌,교차로,교차로부근,20대,60대,북구,동천동
3155,2023123100100047,일요일,대구광역시 달서구 성당동,경상사고,안전운전불이행,젖음/습기,흐림,남,기타불명,승용,...,31,8,차대차,측면충돌,교차로,교차로안,20세 이하,50대,달서구,성당동
3156,2023123100100069,일요일,대구광역시 북구 읍내동,중상사고,안전거리미확보,젖음/습기,맑음,남,상해없음,이륜,...,31,10,차대차,기타,교차로,교차로부근,40대,60대,북구,읍내동
3157,2023123100100085,일요일,대구광역시 동구 신암동,경상사고,안전운전불이행,건조,맑음,남,경상,기타불명,...,31,11,차량단독,공작물충돌,기타,기타,40대,미분류,동구,신암동


In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [24]:
daegu

,사고번호,요일,시군구,사고내용,법규위반,노면상태,기상상태,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,...,day,사고시간대,사고유형_1,사고유형_2,도로형태_1,도로형태_2,가해운전자 연령대,피해운전자 연령대,구,동
0,2021010100100174,금요일,대구광역시 남구 대명동,경상사고,안전운전불이행,건조,맑음,남,상해없음,승용,...,1,18,차대차,추돌,교차로,교차로부근,20대,20대,남구,대명동
1,2021010100100258,금요일,대구광역시 중구 교동,부상신고사고,신호위반,건조,맑음,남,부상신고,승용,...,1,22,차대차,측면충돌,교차로,교차로안,40대,60대,중구,교동
2,2021010200100088,토요일,대구광역시 서구 비산동,경상사고,안전운전불이행,건조,맑음,여,경상,승용,...,2,11,차대차,측면충돌,교차로,교차로안,60대,50대,서구,비산동
3,2021010200100338,토요일,대구광역시 수성구 범물동,부상신고사고,직진우회전진행방해,건조,맑음,남,부상신고,승용,...,2,19,차대차,측면충돌,교차로,교차로안,40대,60대,수성구,범물동
4,2021010200100375,토요일,대구광역시 북구 침산동,경상사고,신호위반,건조,맑음,남,상해없음,보행자,...,2,20,차대사람,횡단중,교차로,교차로부근,20대,20대,북구,침산동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,2023123000100440,토요일,대구광역시 북구 동천동,중상사고,신호위반,젖음/습기,비,남,상해없음,자전거,...,30,20,차대차,측면충돌,교차로,교차로부근,20대,60대,북구,동천동
3155,2023123100100047,일요일,대구광역시 달서구 성당동,경상사고,안전운전불이행,젖음/습기,흐림,남,기타불명,승용,...,31,8,차대차,측면충돌,교차로,교차로안,20세 이하,50대,달서구,성당동
3156,2023123100100069,일요일,대구광역시 북구 읍내동,중상사고,안전거리미확보,젖음/습기,맑음,남,상해없음,이륜,...,31,10,차대차,기타,교차로,교차로부근,40대,60대,북구,읍내동
3157,2023123100100085,일요일,대구광역시 동구 신암동,경상사고,안전운전불이행,건조,맑음,남,경상,기타불명,...,31,11,차량단독,공작물충돌,기타,기타,40대,미분류,동구,신암동


# 라벨인코딩

In [39]:
le = LabelEncoder()

columns_to_encode = ['요일', '시군구', '사고내용', '법규위반', '노면상태', '기상상태',
                     '가해운전자 성별', '가해운전자 상해정도', '피해운전자 상해정도', '피해운전자 차종','피해운전자 성별', '사고유형_1', '사고유형_2',
                     '도로형태_1', '도로형태_2', '가해운전자 연령대', '피해운전자 연령대', '구', '동']

# 각 컬럼에 대해 라벨 인코딩 수행
for column in columns_to_encode:
    daegu[column] = le.fit_transform(daegu[column])

# 원핫인코딩

In [29]:
daegucolumns_to_encode = ['요일', '시군구', '법규위반', '기상상태', '가해운전자 성별', '가해운전자 연령대']

# 원핫 인코딩 적용
df_encoded = pd.get_dummies(daegu, columns=columns_to_encode)

,사고번호,요일,시군구,사고내용,법규위반,노면상태,기상상태,가해운전자 성별,가해운전자 상해정도,피해운전자 차종,...,day,사고시간대,사고유형_1,사고유형_2,도로형태_1,도로형태_2,가해운전자 연령대,피해운전자 연령대,구,동
0,2021010100100174,금요일,대구광역시 남구 대명동,경상사고,안전운전불이행,건조,맑음,남,상해없음,승용,...,1,18,차대차,추돌,교차로,교차로부근,20대,20대,남구,대명동
1,2021010100100258,금요일,대구광역시 중구 교동,부상신고사고,신호위반,건조,맑음,남,부상신고,승용,...,1,22,차대차,측면충돌,교차로,교차로안,40대,60대,중구,교동
2,2021010200100088,토요일,대구광역시 서구 비산동,경상사고,안전운전불이행,건조,맑음,여,경상,승용,...,2,11,차대차,측면충돌,교차로,교차로안,60대,50대,서구,비산동
3,2021010200100338,토요일,대구광역시 수성구 범물동,부상신고사고,직진우회전진행방해,건조,맑음,남,부상신고,승용,...,2,19,차대차,측면충돌,교차로,교차로안,40대,60대,수성구,범물동
4,2021010200100375,토요일,대구광역시 북구 침산동,경상사고,신호위반,건조,맑음,남,상해없음,보행자,...,2,20,차대사람,횡단중,교차로,교차로부근,20대,20대,북구,침산동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,2023123000100440,토요일,대구광역시 북구 동천동,중상사고,신호위반,젖음/습기,비,남,상해없음,자전거,...,30,20,차대차,측면충돌,교차로,교차로부근,20대,60대,북구,동천동
3155,2023123100100047,일요일,대구광역시 달서구 성당동,경상사고,안전운전불이행,젖음/습기,흐림,남,기타불명,승용,...,31,8,차대차,측면충돌,교차로,교차로안,20세 이하,50대,달서구,성당동
3156,2023123100100069,일요일,대구광역시 북구 읍내동,중상사고,안전거리미확보,젖음/습기,맑음,남,상해없음,이륜,...,31,10,차대차,기타,교차로,교차로부근,40대,60대,북구,읍내동
3157,2023123100100085,일요일,대구광역시 동구 신암동,경상사고,안전운전불이행,건조,맑음,남,경상,기타불명,...,31,11,차량단독,공작물충돌,기타,기타,40대,미분류,동구,신암동


In [ ]:
x = daegu.drop(['사고번호', '사고일시', 'EPDO', ], axis = 1)
y = daegu['EPDO']